In [1]:
import os
import csv
import random
import numpy as np
from PIL import Image as PIL_Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR
from torch.autograd import Variable

from torchvision import transforms

os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [2]:
seed = 400

os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)    
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [3]:
device_type = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device_type)
print(device)

cuda


In [4]:
image_path = "../img/"
label_path = "../"

In [5]:
seq_size = 20

In [6]:
str_to_label = {
    "100000" : 0,
    "101000" : 1,
    "110000" : 2,
    "101010" : 3,
    "000000" : 4,
    "010000" : 5,
    "111000" : 6,
    "001000" : 7,
    "100100" : 8,
    "110010" : 9,
    "110100" : 10,
    "101100" : 11,
    "011000" : 12,
}


label_to_str = {}
for k, v in str_to_label.items():
    label_to_str[v] = k

label_counter = [0] * len(str_to_label)
    
print(label_to_str)
print(label_counter)

{0: '100000', 1: '101000', 2: '110000', 3: '101010', 4: '000000', 5: '010000', 6: '111000', 7: '001000', 8: '100100', 9: '110010', 10: '110100', 11: '101100', 12: '011000'}
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:
class KartDataSet(data.Dataset):
    def __init__(self, csv_files):
        self.image_seqs = []
        self.labels = []
        
        for csv_file in csv_files:
            images = []
            labels = []
            
            with open(csv_file) as csvfile: 
                csv_reader = csv.reader(csvfile)
                
                for row in csv_reader:
                    images.append(image_path + row[0])
#                     labels.append(int(row[1], 2))
                    labels.append(str_to_label[row[1]])
                    
            for i in range(len(images)):
                if i + seq_size < len(images):
                    # skip last n images
                    self.image_seqs.append(images[i:i + seq_size])
                    self.labels.append(labels[i + seq_size])
                    label_counter[labels[i + seq_size]] += 1
                    
    def __getitem__(self, index):
        images = [PIL_Image.open(image_path) for image_path in self.image_seqs[index]]
        
        preprocess = transforms.Compose([
            transforms.Resize((90, 160)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
        ])
        
        preprocessed_images = []
        for img in images:
            preprocessed_images.append(preprocess(img))
            
        return torch.stack(preprocessed_images, dim = 1), self.labels[index]
    
    def __len__(self):
        return len(self.image_seqs)

In [8]:
csvs = [label_path + "test.csv"]
dataset = KartDataSet(csvs)

In [9]:
batch_size = 32

In [10]:
loader = data.DataLoader(
    dataset,
    batch_size = batch_size,
    num_workers = 0,
    shuffle = False,
    drop_last = False
)

In [11]:
print(len(loader))
print(len(dataset))

27
847


In [12]:
num_classes = len(str_to_label)

class Cnn3DModel(nn.Module):
    def __init__(self):
        super(Cnn3DModel, self).__init__()
        
        self.conv_layer1 = self._conv_layer_set(3, 16)
        self.conv_layer2 = self._conv_layer_set(16, 32)
        self.conv_layer3 = self._conv_layer_set(32, 64)
        self.conv_layer4 = self._conv_layer_set(64, 128)
        self.fc1 = nn.Linear(3072, 1024)
        self.bn1 = nn.BatchNorm1d(1024)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(1024, num_classes)
    
    def _conv_layer_set(self, in_c, out_c):
        conv_layer = nn.Sequential(
            nn.Conv3d(in_c, out_c, kernel_size = (3, 3, 3), padding = (1, 0, 0)),
            nn.BatchNorm3d(num_features = out_c),
            nn.ReLU(),
            nn.MaxPool3d((2, 2, 2)),
        )
        return conv_layer
    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        
        out = out.view(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.bn1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        
        return out

In [13]:
save_folder = "../model/weights/"
model_name = "model1.pt"
save_path = os.path.join(save_folder, model_name)

model = Cnn3DModel()
model.load_state_dict(torch.load(save_path))
model = model.to(device)

print(model)

Cnn3DModel(
  (conv_layer1): Sequential(
    (0): Conv3d(3, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 0, 0))
    (1): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (conv_layer2): Sequential(
    (0): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 0, 0))
    (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (conv_layer3): Sequential(
    (0): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 0, 0))
    (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (conv_layer4): Sequen

In [15]:
print(label_counter)
normed_weights = [1 - (x / sum(label_counter)) for x in label_counter]
normed_weights = torch.FloatTensor(normed_weights).to(device)
print(normed_weights)

criterion = nn.CrossEntropyLoss(normed_weights)
# optimizer = optim.Adam(model.parameters(), lr = lr, weight_decay = 5e-4)
# scheduler = CosineAnnealingLR(optimizer, T_max = 50)

[437, 150, 135, 33, 5, 21, 20, 18, 11, 7, 5, 4, 1]
tensor([0.4841, 0.8229, 0.8406, 0.9610, 0.9941, 0.9752, 0.9764, 0.9787, 0.9870,
        0.9917, 0.9941, 0.9953, 0.9988], device='cuda:0')


In [18]:
epoch = 0
model.eval()
loss_val = 0
matches = 0
log_interval = 1

with torch.no_grad():
    for idx, train_batch in enumerate(loader):
        inputs, labels = train_batch
        inputs, labels = inputs.to(device), labels.to(device)

        outs = model(inputs)
        preds = torch.argmax(outs, dim = -1)
        loss = criterion(outs, labels)
        
        print("preds : ", preds)
        print("loss : ", loss)

        loss_val += loss.item()
        matches += (labels == preds).sum().item()
        
        print("matches : ", matches)

        if (idx + 1) % log_interval == 0:
            train_loss = loss_val / log_interval
            train_acc = matches / batch_size / log_interval

            print(
                f"Epoch[{epoch + 1}/1]({idx + 1}/{len(loader)}) || ",
                f"training loss {train_loss:4.4} || training accuracy {train_acc:4.2%}")

        loss_val = 0
        matches = 0
        print()

preds :  tensor([4, 4, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
loss :  tensor(0.1789, device='cuda:0')
matches :  29
Epoch[1/1](1/27) ||  training loss 0.1789 || training accuracy 90.62%

preds :  tensor([ 0,  0,  0,  0,  0,  0,  0,  1,  1,  3,  3,  1,  1,  1,  6,  2,  2,  2,
         2,  2,  2, 12,  1,  3,  3,  1,  6,  5,  2,  2,  2,  2],
       device='cuda:0')
loss :  tensor(0.0121, device='cuda:0')
matches :  32
Epoch[1/1](2/27) ||  training loss 0.0121 || training accuracy 100.00%

preds :  tensor([ 2,  5,  1,  1,  1,  0,  2, 10,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
       device='cuda:0')
loss :  tensor(0.0131, device='cuda:0')
matches :  32
Epoch[1/1](3/27) ||  training loss 0.0131 || training accuracy 100.00%

preds :  tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
        0, 0, 1, 0, 0, 0, 0, 0], device